In [1]:
import pandas as pd
import numpy as np
import coremltools

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm

C:\Conda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
TensorFlow version 2.14.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Fail to import BlobReader from libmilstoragepython. No module named 'coremltools.libmilstoragepython'
Fail to import BlobWriter from libmilstoragepython. No module named 'coremltools.libmilstoragepython'


In [2]:
#Load data
fire_df = pd.read_csv('Fire_data.csv')

#######CLEAN DATA##########

# only get rows in Butte, CA
fire_df = fire_df[fire_df['state']=='CA']
fire_df = fire_df[(fire_df['latitude']>=39.0) & (fire_df['latitude']<=41.0)]

# Remove uneccesary rows
fire_df = fire_df[['latitude', 'longitude', 'state', 'disc_clean_date', 'Temp_pre_15', 'Temp_pre_7', 'Wind_pre_15', 'Wind_pre_7','Hum_pre_15', 'Hum_pre_7','Prec_pre_15', 'Prec_pre_7']]
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_15']==-1].index)
fire_df = fire_df.drop(fire_df[fire_df['Prec_pre_7']==-1].index)

# Target row (fire): 1=Fire, 0=no fire
fire_df['fire']=1
wf_df = fire_df[['latitude', 'longitude', 'disc_clean_date']]
wf_df.to_csv('wildfire_data.csv')

In [3]:
weather_df = pd.read_csv('Weather.csv')
weather_df['fire']=0
df = pd.concat([weather_df, fire_df], axis=0)
df = df.sample(frac = 1)
mat=df.corr()
# print(mat['fire'])

In [4]:
X = df[['Temp_pre_7', 'Wind_pre_7', 'Hum_pre_7']]
Y = df[['fire']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
# print(X_test)

In [5]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('No Kernnel')
report = classification_report(y_test,y_pred)
# f1scores.append(accuracy_score(y_test,y_pred, average = 'weighted'))
print(report)

No Kernnel
              precision    recall  f1-score   support

           0       0.90      0.88      0.89        60
           1       0.95      0.96      0.96       145

    accuracy                           0.94       205
   macro avg       0.93      0.92      0.92       205
weighted avg       0.94      0.94      0.94       205



C:\Conda\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
# Convert your scikit-learn model to Core ML format
coreml_model = coremltools.converters.sklearn.convert(svm_model)

# Provide metadata information (optional but recommended)
coreml_model.author = 'Sathya'
coreml_model.license = 'VULCAN'
coreml_model.short_description = 'ML model'

# Save the Core ML model to a file
# coreml_model.save('svm_model.mlmodel')